In [1]:
! pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.6 MB/s eta 0:00:0000:0100:02
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 4.4 MB/s eta 0:00:00a 0:00:01
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488496 sha256=0821d67f625f3da5aa7503a752975937a64095c0923f82261a86d524cad2e610
  Stored in directory: /Users/hongyizhan/Library/Caches/pip/wheels/95/13/41/f7f135ee114175605fb4f0a89e7389f3742aa6c1e1a5bcb657
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

def main():
    spark = SparkSession.builder \
        .appName("SplitDatasetByRow") \
        .master("local") \
        .getOrCreate()

    # Read the entire CSV into a DataFrame
    full_df = spark.read.option("header", "true").csv("/Users/hongyizhan/Desktop/MCGILL/ml insy695/DataCo_Supply_Chain/data/DataCoSupplyChainDataset.csv")

    # Define the size of each batch
    batch_size = 5000  # Change this to the number of rows you want in each batch

    # Calculate the number of batches needed
    total_rows = full_df.count()
    num_batches = total_rows // batch_size + (1 if total_rows % batch_size != 0 else 0)

    # Split the data and process each batch
    for i in range(num_batches):
        # Calculate the start and end indices for each batch
        start_row = i * batch_size
        end_row = start_row + batch_size

        # Create a DataFrame for the batch
        batch_df = full_df.limit(end_row).subtract(full_df.limit(start_row))
        
        # Now you can process each batch_df as needed, such as testing a model
        print(f"Processing batch {i+1} with rows from {start_row + 1} to {end_row}")
        batch_df.show(5)  # Show the first 5 rows of the batch for demonstration

        # Example: Save each batch to a separate file (optional)
        batch_df.write.csv(f"output_batch_{i+1}.csv", header=True)

if __name__ == "__main__":
    main()


The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

/Users/hongyizhan/anaconda3/lib/python3.11/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript
head: illegal line count -- -1


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [3]:
from pyspark.sql import SparkSession
from sklearn.linear_model import SGDClassifier
import numpy as np

def main():
    spark = SparkSession.builder \
        .appName("IncrementalModelTraining") \
        .master("local") \
        .getOrCreate()

    # Read the entire CSV into a DataFrame
    full_df = spark.read.option("header", "true").csv("/Users/hongyizhan/Desktop/MCGILL/ml insy695/DataCo_Supply_Chain/data/DataCoSupplyChainDataset.csv")

    # Define the size of each batch
    batch_size = 2000  # Adjust as needed

    # Calculate the number of batches needed
    total_rows = full_df.count()
    num_batches = total_rows // batch_size + (1 if total_rows % batch_size != 0 else 0)

    # Initialize the classifier
    clf = SGDClassifier(loss='log')  # Using logistic regression; you can change the loss function

    # Loop through each batch and train the model incrementally
    for i in range(num_batches):
        # Calculate the start and end indices for each batch
        start_row = i * batch_size
        end_row = start_row + batch_size

        # Create a DataFrame for the batch
        batch_df = full_df.limit(end_row).subtract(full_df.limit(start_row))
        
        # Convert Spark DataFrame to Pandas DataFrame for scikit-learn
        batch_pd = batch_df.toPandas()
        
        # Assuming your data has features and a label column named 'label'
        # Adjust column names as necessary
        X_batch = batch_pd.drop('label', axis=1).values.astype(np.float32)
        y_batch = batch_pd['label'].values.astype(np.int)

        # Check if it's the first batch to configure the classes
        if i == 0:
            classes = np.unique(y_batch)
            clf.partial_fit(X_batch, y_batch, classes=classes)
        else:
            clf.partial_fit(X_batch, y_batch)

        print(f"Trained batch {i+1}/{num_batches}")

    # You can now use clf to make predictions or evaluate the model
    print("Training complete.")

if __name__ == "__main__":
    main()


The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

/Users/hongyizhan/anaconda3/lib/python3.11/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript
head: illegal line count -- -1


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.